# Analisis de Datos Faltantes sobre un Dataset de Boyas en el Oceano

In [1]:
# Librerias
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
#import session_info
import upsetplot

In [2]:
# linea para correr el notebook utils
%run utils.ipynb

## Cargar el conjunto de Datos desde una Web

In [3]:
# Guardar la url en una variable
datos_url = 'https://github.com/njtierney/naniar/raw/master/data/'

# Almacenar los nombres de los archivos a descargar en una lista
datasets_names = ('oceanbuoys', 'pedestrian', 'riskfactors')

# Almacenar el tipo de extension en una variable
extension = '.rda'

# diccionario que almacena los archivos descargados
datasets_dfs = {} 

for dataset_name in datasets_names:

    dataset_file = f'{ dataset_name }{ extension }' # almacena el nombre de cada archivo con su respectiva extension
    dataset_output_file = f'./data/{ dataset_file }' # ruta y nombre donde se guarda los archivos
    dataset_url = f'{ datos_url }{ dataset_file }' # url de cada archivo a descargar

    !wget -q -O { dataset_output_file } { dataset_url } # linea que descarga los archivos

    # almacenar en el diccionario los archivos descargados
    datasets_dfs[f'{ dataset_name }_df'] = pyreadr.read_r(dataset_output_file).get(dataset_name)

# accede a las llaves del diccionario para verificar que los archivos se hayan cargado
datasets_dfs.keys()

dict_keys(['oceanbuoys_df', 'pedestrian_df', 'riskfactors_df'])

### Incluir el conjunto de datos en nuestro ambiente local

In [4]:
locals().update(**datasets_dfs)
# elimina el diccionario para dejar solamente los dataframes
del datasets_dfs

In [5]:
# carga del dataframe
oceanbuoys_df.shape

(736, 8)

## Analisis Basico de Valores Faltantes sobre el DF

In [8]:
# cantidad de datos
total = 736*8
total

5888

In [9]:
# cantidad de valores completos
v_complet = oceanbuoys_df.missing.number_complete()
v_complet

5711

In [11]:
# cantidad de valores faltantes
v_mising = oceanbuoys_df.missing.number_missing()
v_mising

177

In [16]:
# Porcentaje de valores faltantes en el DF
pct_v = (v_mising/v_complet)*100
pct_v

3.09928208720014

Alrededor del 3% de los datos corresponden a valores faltantes. Aunque no representa un numero amplio, vamos a aplicar todo el analisis requerido.